# Miniproject 2

## Imports and Utilities
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:
import sys
import math
import numpy as np
from collections import namedtuple
# import matplotlib.pyplot as plt

Pose = namedtuple("Pose", ["x", "y", "theta"])
Landmark = namedtuple("Landmark", ["x", "y"])
Measurement = namedtuple("Measurement", ["r", "b"])

def eval_gaussian(x, mean=0, var=1):
  return 1.0/math.sqrt(2*math.pi*var)*math.exp(-0.5*(x-mean)**2/var)

class Simulator:
  def __init__(self):
    self.robot = Pose(0, 0, 0)

    self.motion_noise_covariance = [[1, 0], [0, 1]]
    self.sensor_noise_covariance = [[1, 0], [0, np.deg2rad(5)**2]]

    self.landmarks = [Landmark(-1, -1), Landmark(-1, -1), Landmark(11, 11), Landmark(-1, 11)]

  def reset(self):
    self.robot = Pose(0, 0, 0)

  def simulate_motion(self, delta_p, delta_theta):
    noise = np.random.multivariate_normal([0, 0], self.motion_noise_covariance)
    x = self.robot.x + math.cos(self.robot.theta)*(delta_p + noise[0])
    y = self.robot.y + math.sin(self.robot.theta)*(delta_p + noise[0])
    theta = self.robot.theta + delta_theta + noise[1]
    self.robot = Pose(x, y, theta)

  def simulate_sensing(self):
    measurements = []
    for landmark in self.landmarks:
      noise = np.random.multivariate_normal([0, 0], self.sensor_noise_covariance)

      r = math.hypot(landmark[0]-self.robot.x, landmark[1]-self.robot.y) + noise[0]
      b = math.atan2(landmark[1]-self.robot.y, landmark[0]-self.robot.x) + noise[1]

      measurements.append(Measurement(r, b))

    return measurements

  def stringify(self):
    # For CAT-SOOP internal use only. Please ignore.
    # TODO (Jiayuan):: actually stringify it...
    return 'Simulator()'


def drive_in_a_square(simulator):
  poses = []
  poses.append(simulator.robot)
  for sides in range(4):
    for i in range(100):
      simulator.simulate_motion(.1, 0)
      poses.append(simulator.robot)
    simulator.simulate_motion(0, np.pi/2)
    poses.append(simulator.robot)

  return np.array(poses)

def press(event):
  if event.key == 'q':
    sys.exit(0)
  plt.close()

def draw_samples(samples):
  fig, ax = plt.subplots()
  ax.axis([-2, 12, -2, 12])
  fig.canvas.mpl_connect('key_press_event', press)
  ax.plot(samples[:, 0], samples[:, 1], 'g+')
  plt.show()

def draw_trajectories(true_poses, estimated_poses):
  fig, ax = plt.subplots()
  ax.axis([-20, 20, -20, 20])
  fig.canvas.mpl_connect('key_press_event', press)

  ax.plot(true_poses[:, 0], true_poses[:,1], 'b')
  ax.plot(estimated_poses[:, 0], estimated_poses[:,1], 'r')
  plt.show()




## Problems

### Warmup 1
Let's make sure we know how to resample a set of
weights.

Please write a function that takes in a list of samples and associated weights,
and re-samples from the list of samples with replacement according the
likelihood of each sample given by the list of weights. Please generate 100 samples.

This should be an easy question --- you should feel free to use the sampling
helper functions in numpy. Specifically, take a look at the function numpy.random.choice.


For reference, our solution is **1** lines of code.

In [ ]:
def warmup_1(samples, weights):
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
import random; import numpy.random; random.seed(0); numpy.random.seed(0);assert np.allclose(warmup_1(np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=np.int64), np.array([0.08392318, 0.02899479, 0.10480888, 0.1068318 , 0.13123766, 0.05080644, 0.06685696, 0.11074681, 0.14556308, 0.17023041], dtype=np.float64)), np.array([6, 8, 7, 6, 4, 7, 4, 9, 9, 4, 8, 6, 6, 9, 0, 1, 0, 9, 8, 9, 9, 8, 5, 8, 2, 7, 2, 9, 6, 4, 3, 8, 5, 6, 0, 7, 7, 7, 9, 7, 4, 4, 8, 0, 7, 7, 2, 2, 3, 4, 6, 4, 9, 1, 2, 2, 7, 3, 5, 3, 2, 1, 7, 2, 2, 4, 8, 1, 9, 1, 9, 5, 9, 7, 8, 0, 3, 2, 3, 2, 3, 4, 0, 8, 6, 3, 6, 1, 7, 9, 3, 7, 2, 8, 3, 2, 7, 0, 8, 0], dtype=np.int64), atol=1e-6)
print('Tests passed.')

### Warmup 2
We are going to be sampling from Gaussian
systems in this miniproject. Let us check that we know how to sample from a
multi-variate Gaussian.

Please write a function that takes in a mean vector, a covariance matrix and a
integer number of samples and returns a list of the requested number of
samples drawn from a Gaussian with the given mean and covariance.

This should be an easy question --- you should feel free to use the sampling
helper functions in numpy. Specifically, take a look at the function numpy.random.multivariate_normal.


For reference, our solution is **2** lines of code.

In [ ]:
def warmup_2(mean, covariance, num_samples):
  """Sample from a multi-variate Gaussian distribution.

  Args:
    mean: a vector in numpy.
    covariance: a matrix in numpy.
    num_samples: an integer.

  Returns:
    sample: a numpy vector with length num_samples.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
import random; import numpy.random; random.seed(0); numpy.random.seed(0);assert np.allclose(warmup_2(np.array([ 0.35563307, -0.45998405,  0.47038804], dtype=np.float64), np.array([[2.18405112, 1.00414281, 1.49286069],  [1.00414281, 0.58927147, 0.64943064],  [1.49286069, 0.64943064, 1.09813033]], dtype=np.float64), 5), np.array([[-2.35284352, -1.74151879, -1.13022555],  [-2.79906741, -2.68743248, -1.56968391],  [-1.03394876, -1.07404052, -0.54291808],  [-0.41991666, -0.6895003 ,  0.26060185],  [-0.81622825, -0.99467384, -0.2275568 ]], dtype=np.float64), atol=1e-6)
print('Tests passed.')

### Sampling Question 1
Let us now start by building a motion model for a
quasi-holonomic ground robot.

A common motion model for such a robot assumes that the robot has a pose given
by $[x_t, y_t, \theta_t]$ at time $t$. The robot takes a command $[\Delta p_t,
\Delta \theta_t]$ which modifies the robot's pose accoding to the following:

$$
\begin{bmatrix} x_{t+1} \\ y_{t+1} \\ \theta_{t+1} \end{bmatrix} =
\begin{bmatrix} x_t + \cos(\theta_t) (\Delta p_t + \omega_t) \\
y_t + \sin(\theta_t)(\Delta p_t + \omega_t) \\
\theta_t + \Delta \theta_t + \nu_t
\end{bmatrix}
$$

where $[\omega_t, \nu_t]$ is Gaussian error injected at each time
step, distributed as $\omega_t \sim \mathcal{N}(0, \sigma^2_p)$, and
$\nu_t \sim \mathcal{N}(0, \sigma^2_\theta)$. The units of $\Delta  p_t$ is in
metres and $\Delta \theta_t$ in radians.

Let us assume that the robot's pose at each time step is a random variable
given as a set of samples.

Please implement a function that takes in a set of sampled poses
drawn from prior pose random variable $\mathbf{X}_t$
and a control $[\Delta p_t, \Delta \theta_t]$ and returns a set
of samples drawn from the posterior $X_{t+1}$.

At each step, please use numpy.random.multivariate_normal to sample the noise.


For reference, our solution is **13** lines of code.

In [ ]:
def motion_model(samples, delta_p, delta_theta, simulator):
  """A motion model.

  Args:
    samples: a list of samples
    delta_p: translation command
    delta_theta: rotation command
    simulator: a Simulator object

  Returns:
    a list of new samples by applying the motion model
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
import random; import numpy.random; random.seed(0); numpy.random.seed(0);assert np.allclose(motion_model(np.array([[ 0.14940791,  0.03336743, -0.01790344],  [-0.08540957,  0.03130677, -0.22279039],  [ 0.08644362,  0.06536186, -0.06476612],  [-0.14543657,  0.22697546,  0.00399318],  [ 0.15327792, -0.01871839,  0.12822574],  [ 0.03781625,  0.01549474, -0.07747392],  [-0.03479121, -0.19807965,  0.01364402],  [ 0.12023798,  0.12302907, -0.03380064],  [-0.1048553 , -0.03023028, -0.12391994],  [ 0.19507754, -0.17062702, -0.04447554]], dtype=np.float64), 1, 0, Simulator()), np.array([[ 2.91301728, -0.01611596,  0.38225377],  [ 1.84442332, -0.40589916,  2.01810281],  [ 2.94798951, -0.12022892, -1.042044  ],  [ 1.8046363 ,  0.2347625 , -0.14736402],  [ 1.0426968 ,  0.09595719,  0.53882424],  [ 1.17842815, -0.07305016,  1.37679959],  [ 1.7260826 , -0.17405276,  0.13531904],  [ 1.5632765 ,  0.07423486,  0.29987369],  [ 2.37009854, -0.33850601, -0.32907821],  [ 1.50684678, -0.22900717, -0.89857128]], dtype=np.float64), atol=1e-6)
print('Tests passed.')

### Sampling Question 2
Let us now add in a measurement model. Let us assume that
there are four landmarks located at locations (-1, -1), (11, -1), (11, 11) and
(-1, 11). The robot is equipped with a sensor that can measure the range and
bearing [r, b] from the robot's location to each of these landmarks. Each of
these measurements is corrupted with noise $[q_r, q_b]$ distributed according
to $q_r \sim \mathcal{N}(0, \sigma^2_r)$ and $q_b \sim \mathcal{N}(0,
\sigma^2_b)$.

Given a set of measurements, we can use this model to compute the importance
weights of a set of samples of the robot pose.

Please implement a function that takes a set of samples and measurements and
returns the importance weights of the samples.



For reference, our solution is **18** lines of code.

In [ ]:
def compute_weights(samples, measurement, simulator):
  """Compute the importance weights of the samples.

  Args:
    a list of samples
    a measurement vector
    a simulator object

  Returns:
    a list of importance weights
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
import random; import numpy.random; random.seed(0); numpy.random.seed(0);assert np.allclose(compute_weights(np.array([[ 0.06536186, -0.25529898,  0.07543629],  [ 0.22697546, -0.0742165 , -0.12691735],  [-0.01871839,  0.00457585,  0.13376022],  [ 0.01549474,  0.14693588,  0.03300091],  [-0.19807965, -0.08877857, -0.03036106],  [ 0.12302907,  0.0156349 ,  0.10492744],  [-0.03023028, -0.03873268, -0.09150351],  [-0.17062702, -0.14200179,  0.17023727],  [-0.04380743, -0.05096522, -0.10932702],  [-0.16138978,  0.07774904, -0.01856509]], dtype=np.float64), [Measurement(r=0.5187470011794195, b=-2.3224308778331726), Measurement(r=0.9034084248042221, b=-2.4592240845320847), Measurement(r=15.528166957765391, b=0.8227771705573576), Measurement(r=11.111878239570428, b=1.6878518665472155)], Simulator()), np.array([0.01711671, 0.04172281, 0.14431346, 0.04618495, 0.05960219, 0.16749519, 0.17919444, 0.15658841, 0.18128191, 0.00649993], dtype=np.float64), atol=1e-6)
print('Tests passed.')

### Main Question
Now let's put everything together. Please write a method
localization() that does the following:

  * Creates a simulator.
  * Initialize a set of samples drawn from a Gaussian distribution with mean
equal to the robot starting position, and a diagonal covariance matrix equal
to diag(1, 1, np.deg2rad(5))^2.
  * Drive the robot along the same pattern as in the drive_in_a_square method,
but on each step, after the command motion do the following:
  * use the motion_model method to update the sample between each simulated
motion,
  * use compute_weights to compute the sample weights from the
measurements,
  * resample a new set of samples,
  * blur the samples by applying noise drawn from a zero-mean Gaussian with
covariance equal to `diag(.01, .01, np.deg2rad(1))^2`.


For reference, our solution is **56** lines of code.

In [ ]:
def localization(simulator):
  """Main function for robot localization.

  Args:
    simulator: a Simulator object.

  Returns:
    true_poses
    estimated_poses
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
import random; import numpy.random; random.seed(0); numpy.random.seed(0);ret = localization(Simulator())
target = (np.array([[  0.        ,   0.        ,   0.        ],  [ -1.20652685,   0.        ,   1.65813068],  [ -1.11583177,  -1.03583954,   2.4430882 ],  [ -1.10451676,  -1.04534115,   2.75183944],  [ -2.72188797,  -0.38097931,   1.20182525],  [ -2.53915907,   0.09157921,   2.40352311],  [ -3.768122  ,   1.20938959,   1.46325287],  [ -3.71754809,   1.67784007,   0.47343032],  [ -2.62105182,   2.23956293,  -0.08755068],  [ -3.78292727,   2.34154662,  -0.14904544],  [ -2.47097166,   2.14454468,   0.5173091 ],  [ -1.70834724,   2.5784736 ,   0.53245588],  [ -1.10121932,   2.93620598,   2.20076694],  [ -0.80609504,   2.53141028,   2.90723856],  [ -1.95122747,   2.80480021,   3.71832823],  [ -2.63729568,   2.3585041 ,   5.2945212 ],  [ -1.86548449,   1.18593112,   5.55175652],  [ -1.82190491,   1.14681872,   6.9276981 ],  [ -1.7867864 ,   1.17321316,   9.10069401],  [ -1.72051926,   1.15095217,  10.81948515],  [ -1.78301164,   0.79973721,  10.51805944],  [ -1.96316367,   0.45158787,  10.63296233],  [ -1.66558526,   1.23595258,  11.84530115],  [ -1.35221274,   0.9605101 ,  12.88735611],  [ -1.7564175 ,   0.82611869,  13.03484868],  [ -1.83443929,   0.78663555,  13.23156861],  [ -2.62536818,   0.1661774 ,  13.2059434 ],  [ -2.21888563,   0.46855164,  13.82066027],  [ -2.09364755,   0.85093824,  13.22571488],  [ -2.53504005,   0.50883489,  13.92738561],  [ -2.36664485,   1.29974254,  14.94071691],  [ -3.43830345,   2.33318556,  14.33299126],  [ -3.30987355,   1.68574783,  13.00978597],  [ -2.86500876,   1.89704062,  13.52752377],  [ -4.04661907,   0.20512931,  15.10436098],  [ -2.08965588,  -1.14407588,  15.87461526],  [ -1.67016866,  -1.07351304,  16.7146152 ],  [ -1.10313494,  -0.17735428,  16.70218093],  [ -0.6062058 ,   0.58681182,  15.17814655],  [ -0.58328893,   0.57339008,  14.65021301],  [ -1.52319458,   2.24176882,  14.56708767],  [ -1.50864129,   2.21002946,  14.20664975],  [ -1.54839376,   2.78122799,  13.39328167],  [ -1.69967676,   2.61683274,  15.31779432],  [ -1.87656192,   2.68957719,  15.88897385],  [ -2.33869303,   2.60500083,  16.08672124],  [ -3.71871358,   2.05578954,  14.36416421],  [ -3.88974604,   2.79625956,  12.64898526],  [ -3.74901395,   2.80791262,  12.52751986],  [ -1.65151458,   2.72638216,  13.89420068],  [ -1.19682745,   4.56081211,  14.6922142 ],  [ -0.84873365,   3.99943508,  13.82075353],  [ -0.56860651,   4.85501071,  14.72870211],  [ -0.47180796,   4.71091837,  12.89845804],  [ -1.1108746 ,   4.49053057,  10.39165405],  [ -0.64700009,   5.16290271,   9.85300251],  [ -1.71609893,   4.67488702,  10.00844382],  [ -1.32238166,   4.93490597,  10.27377123],  [ -1.60206666,   4.61717708,  11.62538362],  [ -1.92212079,   5.05631559,  13.1236172 ],  [ -1.37552678,   5.39690949,  12.58339815],  [ -2.1891202 ,   5.38305467,  13.10525727],  [ -3.17173875,   4.79553009,  13.66423989],  [ -3.13995498,   4.85764977,  13.34484374],  [ -3.26823403,   4.7311352 ,  13.35979509],  [ -4.91164229,   3.0611319 ,  15.5512029 ],  [ -5.43456633,   3.1437838 ,  15.90420166],  [ -5.47423765,   3.13589727,  15.80514436],  [ -6.22395243,   3.06280894,  15.42882687],  [ -7.35839359,   3.38796194,  15.65385578],  [ -6.60625486,   3.34722585,  16.04187562],  [ -8.03328793,   2.85218505,  15.18290959],  [ -9.02794108,   3.42837944,  16.3951281 ],  [ -8.73487302,   3.66886546,  16.447787  ],  [ -8.64790393,   3.74824792,  17.16909542],  [ -8.7715928 ,   2.62488529,  17.46150034],  [ -8.68951046,   2.18072264,  16.9070022 ],  [ -9.00765959,   1.36471778,  18.69435476],  [ -7.33370679,   1.1028121 ,  17.19899812],  [ -7.34770782,   0.92764826,  18.64090381],  [ -7.67632624,   0.99722787,  20.63682201],  [ -7.74190809,   1.29544164,  20.51948653],  [ -7.91252345,   3.01085307,  21.43638362],  [ -5.75490274,   1.67381747,  21.63361363],  [ -5.21885796,   1.47355588,  22.3133458 ],  [ -6.08083256,   1.18580315,  23.3510433 ],  [ -6.06079093,   1.27941837,  21.26366509],  [ -4.88490935,   0.23241839,  19.86945927],  [ -3.72744376,   2.11651495,  21.2387438 ],  [ -4.03893783,   2.40810405,  21.31746181],  [ -4.59551005,   2.8524004 ,  21.68287996],  [ -5.93584219,   3.27918831,  22.16593756],  [ -6.2211186 ,   3.22881106,  23.28923172],  [ -5.89777284,   4.38493307,  24.23766771],  [ -5.50660813,   3.89696016,  24.39821938],  [ -6.45432238,   4.7528352 ,  24.72790873],  [ -5.54315663,   4.36239953,  24.76485071],  [ -5.08640187,   4.18634875,  24.40183065],  [ -4.88462288,   4.00544204,  24.63265558],  [ -4.06935898,   3.55997067,  27.12481302],  [ -4.29819359,   4.07064452,  25.05210991],  [ -4.81247048,   4.11220145,  26.15408538],  [ -5.65116205,   2.74257751,  26.44604012],  [ -5.44393495,   3.52948505,  26.88371618],  [ -5.63691108,   4.58889661,  28.13921683],  [ -5.57897242,   4.58102011,  28.26489118],  [ -6.71519238,   4.59174942,  26.69609592],  [ -6.72618373,   3.11476853,  28.39357355],  [ -5.89131111,   3.21479296,  29.23326305],  [ -6.48947418,   2.36234664,  30.16563207],  [ -6.09246734,   1.16635007,  29.64513537],  [ -6.13978262,   0.93293026,  29.88405851],  [ -6.16562397,   1.59641374,  29.62589969],  [ -5.933896  ,   2.63643123,  27.89828568],  [ -7.24487638,   3.1540557 ,  27.66846548],  [ -7.64379009,   3.43041781,  28.72797747],  [ -6.50850792,   3.98393235,  28.73727111],  [ -6.7583715 ,   3.85922235,  27.58901646],  [ -6.18675914,   3.39193262,  27.93749903],  [ -7.38785416,   3.81253165,  28.4922684 ],  [ -9.04978327,   3.44449471,  29.84147358],  [ -9.05852914,   1.0527249 ,  30.67993354],  [-10.05132216,   1.95196556,  31.16874702],  [-11.09522263,   2.21538308,  31.71686424],  [-10.24803923,   2.47831829,  32.95228674],  [-10.25085874,   2.3964741 ,  32.15164301],  [ -9.38854049,   3.17710084,  31.08559428],  [ -8.7066896 ,   2.94329679,  31.22933288],  [ -9.16267309,   3.02938183,  32.83615653],  [ -9.08336115,   3.55215271,  33.68293013],  [ -8.82166671,   3.23905573,  33.84619919],  [ -9.52683528,   3.84678867,  32.54699814],  [ -9.01221411,   4.94069327,  31.60917836],  [ -8.51332848,   5.03832223,  32.40316166],  [ -8.90289304,   4.44831444,  33.05953923],  [ -8.9647899 ,   5.29685198,  32.95163918],  [ -8.99315819,   4.48859436,  34.61488276],  [ -9.15175794,   4.47948652,  36.56792255],  [ -9.21492446,   4.61379748,  36.17560095],  [ -9.24352736,   5.21824525,  36.64593673],  [ -9.65309149,   5.93752648,  38.12815167],  [ -9.58946437,   5.9666333 ,  38.05173526],  [ -8.50418737,   6.36601994,  39.00628889],  [ -8.33367293,   6.99778723,  38.65134088],  [ -8.66424368,   6.53333829,  36.92953671],  [ -8.7197848 ,   6.58714899,  37.15190777],  [ -8.19813079,   6.26932353,  36.899608  ],  [ -7.61412885,   5.66860926,  35.86146995],  [ -7.84624655,   4.81949664,  34.87499933],  [ -6.64097061,   5.21554446,  34.56915187],  [ -4.35790074,   5.24210389,  35.70690252],  [ -4.2870232 ,   5.40021795,  35.68491251],  [ -4.63678604,   4.66378901,  36.05044693],  [ -4.5292523 ,   6.04196092,  38.07368722],  [ -3.35263951,   6.50453005,  38.43311711],  [ -1.86605521,   7.84566176,  37.92390884],  [ -3.43938694,   7.48590098,  36.66712209],  [ -3.18168368,   7.05482107,  37.11061897],  [ -3.22465038,   7.08349597,  37.23719297],  [ -2.40771541,   6.67679229,  35.70210364],  [ -3.0287904 ,   5.30891847,  36.3099695 ],  [ -2.98467698,   5.06875239,  34.67841252],  [ -4.65933065,   4.86530583,  34.29561724],  [ -5.02825541,   4.96419948,  35.25835665],  [ -4.64996425,   5.28337167,  35.0117051 ],  [ -6.89419221,   4.18767626,  33.48362361],  [ -7.38316261,   5.08936143,  35.65708853],  [ -8.22294535,   3.44114647,  35.25769587],  [ -6.69592389,   4.72780017,  36.2460583 ],  [ -6.81490073,   5.73360723,  36.21980681],  [ -6.84463081,   6.05764991,  35.85039186],  [ -7.15270892,   4.97783964,  33.85428717],  [ -7.39459873,   5.18292071,  35.31521536],  [ -7.82711037,   4.77373181,  36.62536715],  [ -7.93918425,   4.9803271 ,  36.79347139],  [ -8.16771183,   5.27166784,  38.10687645],  [ -8.36950247,   5.1844991 ,  37.41694148],  [ -8.32795506,   5.17245428,  37.62891425],  [ -6.88746601,   5.07116899,  37.80430409],  [ -6.78868514,   5.08159847,  38.76786739],  [ -7.21591148,   4.30334503,  38.63167734],  [ -7.00223897,   4.59139197,  37.96983386],  [ -6.99977882,   4.59207475,  37.6606858 ],  [ -8.47692141,   4.64886345,  38.62984911],  [ -8.44454425,   4.69234399,  38.70423749],  [ -8.67253919,   4.33322753,  39.5295677 ],  [ -9.03164546,   5.68499413,  39.47039678],  [ -8.94318025,   5.24967407,  41.26856644],  [ -8.40335513,   5.49585364,  41.00029923],  [ -7.95898671,   5.56738081,  40.61984428],  [ -7.95007668,   5.56538031,  39.14153797],  [ -7.76210344,   7.02163394,  39.39153367],  [ -7.99699673,   8.94338482,  38.46991493],  [ -8.70293198,   8.25776092,  40.515484  ],  [ -7.67423563,   7.9108914 ,  40.88653669],  [ -9.46473907,   7.8287712 ,  41.11556532],  [ -9.21442255,   7.89936005,  40.32161667],  [ -8.85977463,   7.69673151,  37.90012619],  [ -8.71362013,   7.72651288,  38.01831995],  [ -8.36219141,   7.84266386,  38.11080695],  [ -8.12326677,   7.94698982,  36.7896408 ],  [ -8.08278103,   7.89496702,  35.86371313],  [ -8.11159265,   7.78863369,  36.50535174],  [ -8.23515619,   8.10067844,  37.4284362 ],  [ -8.3007729 ,   8.11888614,  36.75603843],  [ -8.39437574,   8.24788089,  35.56082667],  [ -9.53648888,   6.45613805,  36.32101761],  [ -9.49152439,   6.22569669,  36.57993751],  [ -8.92165928,   5.0508554 ,  35.66096752],  [ -9.10930358,   4.67901309,  35.09966491],  [ -8.41583425,   5.0967244 ,  34.53277925],  [ -9.30846692,   5.11881259,  34.75619532],  [ -9.50200537,   5.07984708,  33.64890079],  [ -8.61042549,   3.93621035,  33.00523845],  [ -8.59291013,   2.99033971,  33.08859332],  [ -8.61262599,   3.1832083 ,  33.16780485],  [ -8.72715707,   3.80876182,  33.59395214],  [ -9.88896902,   5.48090855,  32.01276502],  [ -9.50203631,   5.74383148,  31.45340806],  [-10.09004616,   5.72178165,  30.81339769],  [-11.03500314,   6.3717757 ,  32.03973725],  [-10.49981106,   6.75694147,  33.61593018],  [-10.50656983,   6.76622676,  34.34324227],  [ -9.81668608,   6.61609579,  36.17003887],  [ -9.77261176,   5.56036222,  36.2600998 ],  [ -9.73693774,   5.29123099,  34.59209619],  [ -8.6800414 ,   5.32778986,  32.97695152],  [ -8.67784192,   5.55287779,  31.98125354],  [ -8.1518527 ,   5.88656291,  30.1577624 ],  [ -8.0404981 ,   5.54205954,  31.10672781],  [ -6.58434907,   5.07690111,  30.11744562],  [ -6.46811977,   4.66068533,  29.90371195],  [ -6.51751788,   5.50295451,  29.7344307 ],  [ -6.41300916,   6.4431708 ,  29.71239213],  [ -6.53829894,   5.50483363,  29.08076021],  [ -5.88382089,   6.18743209,  29.24892428],  [ -5.90795819,   6.15186184,  28.9714932 ],  [ -6.19981579,   5.90744746,  29.36568254],  [ -6.35332827,   5.61217949,  31.80191787],  [ -5.93271857,   5.78310543,  32.16203643],  [ -6.16139844,   5.57172372,  30.99921732],  [ -5.66826674,   5.3534482 ,  29.67358295],  [ -5.65553339,   5.42694511,  30.67443436],  [ -7.10126745,   6.75099104,  30.91771888],  [ -7.50746495,   6.97195334,  32.20462794],  [ -6.84459492,   7.63921712,  33.48448317],  [ -6.52552979,   7.05205047,  33.15578885],  [ -6.47435594,   6.75225422,  32.15080562],  [ -6.36706774,   6.84921515,  31.86336245],  [ -5.81235239,   7.11542115,  33.88315363],  [ -4.09404937,   5.74183627,  33.99989592],  [ -4.52522057,   6.01073338,  31.97154212],  [ -5.78693113,   5.22738971,  30.46831321],  [ -5.61515869,   4.98839373,  30.03076477],  [ -5.57645949,   4.78232408,  28.9326344 ],  [ -6.88578946,   3.76970745,  29.49760322],  [ -6.92844838,   3.65193948,  30.86588591],  [ -6.38269349,   3.31730381,  31.36458209],  [ -6.73459258,   3.33538777,  30.50488199],  [ -6.57725579,   3.13255765,  30.61372552],  [ -7.08864692,   3.66142981,  29.76399353],  [ -7.06979893,   3.89321908,  30.26884981],  [ -6.8023752 ,   3.30031619,  30.92941222],  [ -5.97086672,   2.86051551,  30.31085283],  [ -6.1268345 ,   3.17083957,  29.7464569 ],  [ -6.24815527,   1.94531286,  28.05043688],  [ -6.41171129,   1.982557  ,  29.73242984],  [ -6.3974008 ,   2.1089971 ,  27.72796682],  [ -6.7849285 ,   2.34465958,  26.68828844],  [ -6.77850425,   2.7659139 ,  25.42183465],  [ -7.93257081,   2.42266479,  25.48400678],  [ -7.62517455,   2.53531425,  26.33401127],  [ -7.48047226,   2.90891472,  25.92907045],  [ -7.38285006,   3.00869482,  25.28255525],  [ -5.61922516,   3.27490517,  26.98835266],  [ -5.23957744,   1.97818271,  27.95285173],  [ -6.03273477,   2.24233196,  25.73613265],  [ -5.68689651,   2.4806585 ,  25.48786555],  [ -5.26072599,   2.63870237,  26.39635585],  [ -4.92375401,   3.70095877,  27.21686825],  [ -5.0271817 ,   3.88442586,  26.80716929],  [ -4.95866062,   3.22559672,  27.59760224],  [ -5.79092183,   3.89412714,  27.28573889],  [ -6.16604195,   4.46394206,  26.40460091],  [ -6.30977679,   3.99753026,  24.77937694],  [ -6.76972182,   4.16717903,  24.25112464],  [ -7.08746397,   4.55280808,  21.66463255],  [ -5.76683844,   4.10559572,  22.52015935],  [ -6.60108076,   3.61790687,  22.61788953],  [ -5.98601565,   4.06329703,  23.00648052],  [ -5.78184247,   4.39226425,  23.7087447 ],  [ -5.63162843,   3.37636748,  22.21989531],  [ -4.74283461,   3.58329804,  24.79913012],  [ -5.0700875 ,   3.69671232,  24.08005974],  [ -5.11427299,   3.77422239,  22.98786106],  [ -4.87630673,   4.14216667,  23.60534868],  [ -4.8208837 ,   2.86605152,  24.13995269],  [ -5.18433135,   3.423209  ,  24.76504147],  [ -6.2110426 ,   3.81871792,  25.01299386],  [ -5.09996482,   3.68502967,  23.5544859 ],  [ -5.09251663,   4.68356134,  23.9184215 ],  [ -5.05633098,   4.58638881,  24.04977587],  [ -4.48067582,   3.50148592,  24.16279471],  [ -5.58661751,   5.11509219,  25.2541203 ],  [ -5.4218833 ,   5.13518626,  25.86244266],  [ -5.10001606,   5.42305788,  27.21446221],  [ -5.20787003,   5.61545729,  28.55904169],  [ -5.78511527,   5.44652164,  27.78049825],  [ -6.83235946,   6.01027952,  28.18679097],  [ -7.02287915,   6.02700091,  29.05383429],  [ -7.36798386,   5.68594307,  28.34071495],  [ -7.34424406,   5.68752126,  28.19012739],  [ -7.35529044,   5.68845364,  27.36783706],  [ -7.26723938,   5.57600266,  28.74316462],  [ -6.89376708,   5.76516481,  29.69586311],  [ -6.97773432,   5.20681876,  30.95020649],  [ -8.84598176,   6.1457873 ,  31.34075938],  [ -8.18630381,   6.09610758,  30.54306527],  [ -8.57327423,   6.55746518,  30.67223374],  [ -7.27684576,   5.36490128,  30.82266775],  [ -7.42189276,   5.4627044 ,  29.76093647],  [ -7.42877214,   5.38118856,  30.43208008],  [ -7.75182721,   5.86687821,  29.97978841],  [ -7.51676236,   4.13180131,  29.82836246],  [ -7.50457118,   4.85879475,  29.11062751],  [ -6.81573239,   5.62157839,  28.30090151],  [ -7.17873774,   5.61193193,  28.80635465],  [ -7.74840581,   5.27660551,  28.54426487],  [ -7.39246821,   5.3750877 ,  27.46164557],  [ -7.17985611,   5.15054255,  27.84875217],  [ -7.19283212,   5.15642437,  28.96652311],  [ -8.52522864,   4.05183668,  29.83665499],  [ -8.51760786,   4.95099898,  31.25949727],  [ -7.62339672,   4.80996594,  30.38130597],  [ -7.79917987,   5.10577609,  30.6028829 ],  [ -7.23708348,   4.51170887,  31.28536945],  [ -6.12353795,   4.36549593,  30.12176453],  [ -6.35956536,   5.19683053,  29.45407283],  [ -6.63127593,   4.53780514,  28.90785906],  [ -6.39903226,   4.70839458,  29.71630471],  [ -6.47230893,   4.14273906,  28.95734238],  [ -7.79500304,   3.06652982,  27.70346182],  [ -8.0264521 ,   3.21516631,  29.11315862],  [ -7.98003764,   3.26682549,  29.52094914],  [ -7.93078109,   3.41340661,  29.66250227],  [ -7.98923392,   3.09690784,  28.72045594],  [ -8.20162148,   2.99532687,  29.72024769],  [ -8.38651165,   1.52251852,  28.77943672],  [ -7.9383972 ,   1.77030198,  29.15555835],  [ -7.7768959 ,   1.96615215,  29.7489924 ],  [ -7.68116801,   2.95881866,  30.82696534],  [ -8.31795625,   3.38422644,  31.04762432],  [ -8.63067964,   3.50490991,  30.87052338],  [ -8.59178316,   3.4813076 ,  30.79005385],  [ -6.71454089,   2.12441393,  32.24340061],  [ -5.371546  ,   3.58546114,  32.49984464],  [ -5.98266815,   2.43106064,  32.7799923 ],  [ -5.89517487,   2.84823792,  31.58953905],  [ -8.60207065,   2.37350766,  31.4418356 ],  [ -9.38949869,   2.35310157,  32.3873324 ],  [ -8.47383708,   3.69327259,  30.41840469],  [ -8.82428945,   4.23610628,  30.91978613],  [ -9.29208406,   4.48932422,  31.34535411],  [ -8.91005564,   4.4623187 ,  31.86395041],  [ -8.81093509,   4.50995803,  32.94930375],  [ -8.77670683,   5.42425762,  32.18362496],  [ -8.2025053 ,   5.97848419,  31.86533796],  [ -7.56213663,   6.28735279,  30.38138654],  [ -7.38215243,   5.98452862,  30.8101199 ],  [ -8.37641678,   6.67325086,  30.76106338],  [ -9.83577296,   7.79390005,  29.90319446],  [ -9.90638336,   9.00860642,  30.00437819],  [ -9.7776063 ,   8.20679644,  29.33097561],  [ -9.22327495,   9.18821619,  29.53390605],  [ -8.90468564,  10.178615  ,  29.56088058],  [ -8.7425262 ,  10.73364953,  31.06993712],  [ -8.10082522,  10.50232266,  31.36395918],  [ -8.57558461,  10.52701688,  30.96136035],  [ -7.39375911,   9.94945968,  31.61992997],  [ -7.20179248,   9.989174  ,  30.51869639],  [ -6.90465019,   9.61685032,  30.94666806],  [ -5.50559504,   8.90748265,  31.39434486],  [ -4.42688017,   8.88419856,  31.64728958],  [ -4.41377668,   8.8872855 ,  30.82320594],  [ -4.46449385,   8.92144367,  30.93798966],  [ -5.41646892,   9.41455852,  29.13152719],  [ -4.57998484,  10.38068404,  29.18950362],  [ -5.01718372,   9.8122443 ,  29.03436626],  [ -5.56862164,   9.28809533,  28.52911062],  [ -6.11523401,   9.1457376 ,  27.28220566],  [ -6.1776459 ,   9.24127578,  28.21398203],  [ -8.58205422,   9.38656271,  28.83822975],  [ -9.97450025,   8.50599369,  29.15058018],  [-10.29288613,   8.12378437,  29.32468437],  [ -9.55673314,   9.40817392,  29.21111906],  [ -9.76342644,   9.12706422,  27.12463576],  [ -9.44891375,   8.42485519,  24.95944387],  [ -7.54668918,   8.09186455,  24.91741257],  [ -6.20569746,   7.79856341,  26.04982939],  [ -6.23193885,   7.76430894,  26.05736679],  [ -4.60104946,   9.9267768 ,  24.77934295],  [ -4.26587005,   9.80313435,  24.99841968],  [ -3.67896979,   9.72382344,  25.43029836],  [ -3.628133  ,   9.73941314,  24.87594435],  [ -3.71757775,   9.76290085,  27.03473368]], dtype=np.float64), np.array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],  [-1.18962416e+00,  5.08318034e-02, -5.48998130e-02],  [-1.51384466e+00, -1.12909373e+00,  9.31287262e-01],  [-1.68936814e+00, -1.30002154e+00,  6.68349314e-01],  [-1.74531679e+00, -7.66296634e-01,  1.96143918e+00],  [-2.51619419e+00,  1.55777095e-01,  2.24231372e+00],  [-3.01080787e+00,  6.49072963e-01,  2.35370610e+00],  [-3.75713071e+00,  1.81919897e+00,  2.14260587e+00],  [-2.47452132e+00,  2.10932729e+00,  1.11933129e+00],  [-3.32642746e+00,  2.26802322e+00,  3.42949736e-01],  [-2.79029770e+00,  2.40149843e+00,  4.49945235e-01],  [-1.92231994e+00,  2.31088745e+00,  4.30969240e-01],  [-7.40118130e-01,  2.35125951e+00,  5.83455623e-01],  [-7.34806097e-01,  2.26986597e+00,  4.99411146e-01],  [-1.96746101e+00,  2.36110306e+00,  1.17355601e+00],  [-2.73651978e+00,  2.40742861e+00,  3.19331543e+00],  [-2.38442927e+00,  1.80434714e+00,  1.09300750e+00],  [-2.17513215e+00,  1.82169762e+00,  1.14112109e+00],  [-1.94976006e+00,  1.74303499e+00,  1.11277275e+00],  [-1.87515451e+00,  1.43130596e+00,  2.17982938e+00],  [-1.65189894e+00,  8.19564968e-01, -1.21939221e-01],  [-1.90543338e+00,  4.50754434e-01,  2.83156172e-02],  [-1.65370428e+00,  1.44719849e+00, -9.97072808e-01],  [-1.35122130e+00,  1.80818613e+00, -7.53885098e-01],  [-2.11269439e+00,  1.29721457e+00,  9.52319120e-01],  [-2.08667373e+00,  1.16818408e+00,  1.03937985e+00],  [-2.58333841e+00, -1.07496260e-02,  1.29276986e+00],  [-2.36113703e+00,  9.75234238e-01,  2.27831011e+00],  [-2.09330063e+00,  8.72514805e-01,  2.31523026e+00],  [-1.91056082e+00, -9.40101559e-02, -6.70987939e-01],  [-2.29277706e+00,  1.49451571e+00,  3.26190380e+00],  [-2.81985536e+00,  1.07877728e+00,  3.60197426e+00],  [-2.66187663e+00,  1.61980258e+00,  2.72847233e+00],  [-3.91826197e+00,  2.26442164e+00,  3.10077735e+00],  [-4.85748428e+00,  1.30008142e+00,  3.86735801e+00],  [-5.65028863e+00,  1.20456366e+00,  6.36726669e+00],  [-5.00280067e+00, -8.77334890e-01,  2.59661482e+00],  [-6.93909439e+00,  6.16911040e-01,  2.55530570e+00],  [-4.77006719e+00,  2.08427543e+00,  4.77713604e+00],  [-4.90194138e+00,  4.11818788e+00,  5.28315497e+00],  [-2.72915789e+00,  2.36235795e+00,  4.44690998e+00],  [-2.52731142e+00,  3.10472198e+00,  4.45183232e+00],  [-1.26569290e+00,  3.23553446e+00,  5.21612761e+00],  [-1.67600388e+00,  3.04388320e+00,  5.80626738e+00],  [-1.52048799e+00,  2.67801140e+00,  5.89876630e+00],  [-2.62109631e+00,  2.49820442e+00,  3.38958473e+00],  [-3.48301165e+00,  2.52912542e+00,  3.67668492e+00],  [-3.66430078e+00,  2.68494314e+00,  3.28821084e+00],  [-4.20183733e+00,  2.58196931e+00,  3.55782212e+00],  [-2.70875690e+00,  3.70877680e+00,  3.70124986e+00],  [-1.06666433e+00,  4.73750198e+00,  3.35103907e+00],  [-9.45663911e-01,  4.49834631e+00,  3.14892640e+00],  [-6.54775016e-01,  4.64425878e+00,  3.31976026e+00],  [-6.39127424e-01,  5.16376610e+00,  3.04503798e+00],  [-1.36494033e+00,  4.92212025e+00,  3.88888852e+00],  [-5.98729055e-01,  4.68558840e+00,  4.90910553e+00],  [-1.76136716e+00,  5.35466432e+00,  3.48196080e+00],  [-1.35701230e+00,  5.56100243e+00,  4.41643324e+00],  [-1.87360121e+00,  4.98068237e+00,  3.62940121e+00],  [-2.31841143e+00,  4.89899291e+00,  3.05647457e+00],  [-1.18623963e+00,  4.77205086e+00,  5.16828144e+00],  [-2.54384960e+00,  5.42316970e+00,  5.51426531e+00],  [-2.57903012e+00,  5.71228964e+00,  5.47060785e+00],  [-3.02781695e+00,  5.29446883e+00,  5.86641863e+00],  [-3.16719048e+00,  4.84232498e+00,  5.83244622e+00],  [-4.17840325e+00,  3.04583641e+00,  6.12554663e+00],  [-4.35690158e+00,  2.02573428e+00,  9.85383044e+00],  [-5.03919466e+00,  3.20768642e+00,  8.03436527e+00],  [-5.93703131e+00,  2.85599891e+00,  1.09034427e+01],  [-7.15347174e+00,  3.40916830e+00,  1.81932445e+00],  [-6.55683995e+00,  3.08667484e+00,  4.56533663e+00],  [-7.23581961e+00,  3.15779158e+00,  9.63782599e-01],  [-9.16997541e+00,  3.37111596e+00,  3.00360570e+00],  [-8.82128408e+00,  3.55905665e+00,  3.34661833e+00],  [-8.34218550e+00,  4.07031099e+00,  3.35251129e+00],  [-8.84026391e+00,  3.08890658e+00,  4.39637929e+00],  [-9.17308623e+00,  2.69420398e+00,  4.15420060e+00],  [-8.90454999e+00,  1.28333824e+00,  4.68501568e+00],  [-7.23793112e+00,  1.41826432e+00,  1.12890764e+01],  [-7.02016066e+00,  1.46160130e+00,  1.15709869e+01],  [-6.99114659e+00,  7.64769604e-01,  1.11386233e+01],  [-7.01406330e+00,  1.91150663e+00,  1.18313866e+01],  [-7.25001982e+00,  1.92886338e+00,  1.12046174e+01],  [-6.61712724e+00,  1.60349586e+00,  1.15150901e+01],  [-5.47148213e+00,  1.71930297e+00,  1.24068681e+01],  [-6.18164117e+00,  1.59469302e+00,  1.26623876e+01],  [-6.18989150e+00,  1.16688493e+00,  1.26989445e+01],  [-4.72515314e+00, -8.13459836e-02,  1.22145598e+01],  [-3.77363739e+00,  5.71411540e-01,  1.29971871e+01],  [-2.61641322e+00,  1.12392951e+00,  1.26133294e+01],  [-3.59514565e+00,  1.40741394e+00,  1.33543565e+01],  [-4.53269466e+00,  3.03588579e+00,  1.20747382e+01],  [-5.79431965e+00,  2.89966584e+00,  1.17186774e+01],  [-5.48955674e+00,  4.18674212e+00,  1.17691017e+01],  [-5.34692942e+00,  3.75358579e+00,  1.18186641e+01],  [-6.41393423e+00,  4.68157716e+00,  1.14327526e+01],  [-5.68452898e+00,  4.64703556e+00,  1.22973230e+01],  [-4.73795441e+00,  4.58924511e+00,  1.20142981e+01],  [-4.48911875e+00,  3.59093769e+00,  1.12073214e+01],  [-4.70096752e+00,  3.78833233e+00,  1.09515831e+01],  [-4.31509960e+00,  3.63576100e+00,  1.14941017e+01],  [-5.00249523e+00,  3.90256672e+00,  1.29910845e+01],  [-5.57741691e+00,  3.32530359e+00,  1.24673375e+01],  [-5.87411083e+00,  3.27082095e+00,  1.24095983e+01],  [-5.97779696e+00,  4.62023181e+00,  1.23389772e+01],  [-5.63896393e+00,  4.84971838e+00,  1.40201591e+01],  [-6.61252619e+00,  5.25821261e+00,  1.38695707e+01],  [-6.80780975e+00,  3.94017939e+00,  1.04503194e+01],  [-6.42948062e+00,  3.68213990e+00,  9.24615222e+00],  [-5.78755083e+00,  1.88987500e+00,  9.02975122e+00],  [-6.21214549e+00,  1.07765266e+00,  1.01522202e+01],  [-6.46784530e+00,  1.00247038e+00,  1.04130544e+01],  [-5.98866129e+00,  1.36353701e+00,  1.01879028e+01],  [-6.12380385e+00,  2.38954358e+00,  8.96413277e+00],  [-6.57737700e+00,  2.80283175e+00,  8.47459494e+00],  [-7.87338188e+00,  3.14807157e+00,  1.04434039e+01],  [-7.24075557e+00,  3.32626535e+00,  1.10422967e+01],  [-6.82298947e+00,  3.35798400e+00,  1.14685429e+01],  [-6.06468023e+00,  3.65311139e+00,  1.15006250e+01],  [-6.93806911e+00,  4.01132043e+00,  1.09629058e+01],  [-8.29657030e+00,  3.57662213e+00,  9.39510999e+00],  [-9.63913145e+00,  2.03048572e+00,  5.79202777e+00],  [-9.74380751e+00,  1.99204429e+00,  6.07473438e+00],  [-1.08916174e+01,  2.05043911e+00,  5.16540802e+00],  [-9.43367801e+00,  2.04694781e+00,  5.35522978e+00],  [-1.01502741e+01,  2.27494341e+00,  6.76830266e+00],  [-9.53663660e+00,  2.42819027e+00,  6.13332936e+00],  [-9.44446140e+00,  2.52503691e+00,  6.55305960e+00],  [-8.03194339e+00,  3.12191708e+00,  7.59169786e+00],  [-8.67930004e+00,  3.69251562e+00,  7.10531578e+00],  [-8.78510440e+00,  3.36883475e+00,  6.48017055e+00],  [-9.92817377e+00,  3.54224287e+00,  6.05642907e+00],  [-9.74694354e+00,  4.83746526e+00,  6.25843143e+00],  [-8.47888889e+00,  4.94580816e+00,  6.06753881e+00],  [-9.00159682e+00,  4.27194880e+00,  6.36322029e+00],  [-8.73416134e+00,  5.20835291e+00,  7.02233753e+00],  [-8.71792293e+00,  5.23291851e+00,  7.33940799e+00],  [-8.98118919e+00,  5.20254300e+00,  7.11289797e+00],  [-8.96786881e+00,  4.51848830e+00,  6.75037475e+00],  [-8.93237411e+00,  4.97120209e+00,  6.48701667e+00],  [-9.80428210e+00,  5.50110185e+00,  6.59372119e+00],  [-9.74893993e+00,  5.94588892e+00,  6.58885596e+00],  [-9.04963913e+00,  6.47828552e+00,  6.93958515e+00],  [-9.00469369e+00,  6.89962047e+00,  7.06739046e+00],  [-8.65949140e+00,  6.88679816e+00,  6.96405850e+00],  [-8.65451776e+00,  6.81865244e+00,  7.39070887e+00],  [-8.53549550e+00,  6.82955100e+00,  7.62232294e+00],  [-8.04920149e+00,  6.18877624e+00,  7.26233915e+00],  [-7.84029055e+00,  5.35841800e+00,  7.21991665e+00],  [-7.07914175e+00,  5.11831909e+00,  7.57847804e+00],  [-4.92780166e+00,  5.90752696e+00,  5.76713399e+00],  [-4.70583943e+00,  4.69310813e+00,  5.93720283e+00],  [-4.37661172e+00,  3.75411300e+00,  5.60021787e+00],  [-4.35465417e+00,  5.58240191e+00,  4.20044515e+00],  [-3.67591589e+00,  6.46045207e+00,  3.73886065e+00],  [-1.85277518e+00,  7.41518268e+00,  1.91397970e+00],  [-2.68314691e+00,  8.44652383e+00,  2.60178385e+00],  [-3.02913882e+00,  7.19418247e+00,  2.00872990e+00],  [-3.39086444e+00,  7.42594493e+00,  1.82595899e+00],  [-3.06728330e+00,  6.31553635e+00,  1.84264150e+00],  [-2.72408063e+00,  5.38239355e+00,  2.38131906e+00],  [-3.17159936e+00,  4.78227247e+00,  2.25926497e+00],  [-4.43959401e+00,  5.41895312e+00,  4.26208286e+00],  [-5.07037891e+00,  4.92001678e+00,  4.04650573e+00],  [-4.71526536e+00,  4.75495969e+00,  3.97466912e+00],  [-6.02577046e+00,  4.42347486e+00,  5.29182485e+00],  [-6.89434642e+00,  4.85225373e+00,  5.61530785e+00],  [-7.65036475e+00,  3.16276851e+00,  4.59240190e+00],  [-7.51247056e+00,  4.30230083e+00,  4.86615676e+00],  [-7.20978054e+00,  5.14001993e+00,  4.93529989e+00],  [-7.52551889e+00,  5.59727616e+00,  5.10069334e+00],  [-7.33441445e+00,  5.59480253e+00,  4.95410889e+00],  [-7.97327352e+00,  5.51764887e+00,  5.24961822e+00],  [-7.98561919e+00,  5.71219697e+00,  5.68994880e+00],  [-7.92007845e+00,  5.63426632e+00,  5.54627885e+00],  [-8.05924563e+00,  5.52922735e+00,  5.77506783e+00],  [-8.43584189e+00,  4.57182525e+00,  5.83380638e+00],  [-8.94655447e+00,  5.23177381e+00,  4.95973059e+00],  [-7.92839893e+00,  5.41126575e+00,  5.09833578e+00],  [-6.87308295e+00,  5.92879379e+00,  4.38406408e+00],  [-7.62082112e+00,  4.70292868e+00,  4.20166342e+00],  [-7.30730839e+00,  4.99238900e+00,  3.62243196e+00],  [-7.71497634e+00,  4.50129129e+00,  5.21957624e+00],  [-8.10821801e+00,  4.13414724e+00,  5.71920268e+00],  [-7.98596144e+00,  4.36413265e+00,  5.54941636e+00],  [-8.71720038e+00,  4.73920845e+00,  6.15806439e+00],  [-8.77685845e+00,  5.57584304e+00,  5.76582941e+00],  [-8.97881176e+00,  5.05440474e+00,  5.66056602e+00],  [-8.72378814e+00,  5.07779071e+00,  6.43195086e+00],  [-8.26846902e+00,  5.21482419e+00,  6.40129376e+00],  [-8.32329135e+00,  5.13754407e+00,  6.86962930e+00],  [-7.77849603e+00,  6.21859288e+00,  6.45744873e+00],  [-8.04070875e+00,  7.91274627e+00,  7.01198399e+00],  [-8.38298969e+00,  7.74605827e+00,  6.60534628e+00],  [-7.56843588e+00,  8.39899539e+00,  6.27872040e+00],  [-8.69076978e+00,  7.91677082e+00,  6.63990628e+00],  [-8.89562721e+00,  7.81231662e+00,  7.25759541e+00],  [-9.24317157e+00,  6.95333764e+00,  8.14257306e+00],  [-8.86386794e+00,  7.05802052e+00,  8.99881102e+00],  [-8.37720832e+00,  7.38943288e+00,  8.89728866e+00],  [-8.09431032e+00,  7.63433718e+00,  9.29228054e+00],  [-7.82954201e+00,  7.56694050e+00,  7.26812494e+00],  [-8.27238885e+00,  7.51425078e+00,  8.46773157e+00],  [-8.42325894e+00,  8.21521668e+00,  6.92075747e+00],  [-8.40081602e+00,  8.21201349e+00,  7.54754157e+00],  [-8.37286348e+00,  8.38046116e+00,  8.42933600e+00],  [-8.50578785e+00,  6.36730165e+00,  3.45493529e+00],  [-9.63837053e+00,  6.07090222e+00,  3.50178543e+00],  [-9.64705841e+00,  6.02563982e+00,  3.37980303e+00],  [-9.97501889e+00,  4.68442637e+00,  4.56629995e+00],  [-9.07051201e+00,  5.48842040e+00,  4.79147720e+00],  [-9.33566615e+00,  5.05885044e+00,  4.61751071e+00],  [-9.33423213e+00,  4.77714717e+00,  4.89514204e+00],  [-9.04738538e+00,  4.09919000e+00,  4.68546088e+00],  [-8.85994974e+00,  3.67204769e+00,  4.47033310e+00],  [-8.56486790e+00,  3.44572757e+00,  4.07723435e+00],  [-8.73088024e+00,  3.18562110e+00,  3.70590027e+00],  [-9.68536932e+00,  4.44565819e+00,  4.35069268e+00],  [-9.16945523e+00,  5.41736638e+00,  3.63454772e+00],  [-9.30288221e+00,  5.94074400e+00,  5.20186792e+00],  [-1.05442940e+01,  6.18776730e+00,  6.24046523e+00],  [-9.97442025e+00,  6.49807717e+00,  4.44095602e+00],  [-1.06652792e+01,  6.40928868e+00,  7.91671256e+00],  [-9.67506641e+00,  7.12397712e+00,  6.85177436e+00],  [-9.07259349e+00,  6.17689523e+00,  7.69290782e+00],  [-9.51093455e+00,  5.58317282e+00,  7.71686217e+00],  [-9.40722808e+00,  4.67242204e+00,  7.73613209e+00],  [-8.33869388e+00,  5.71972382e+00,  9.73489750e+00],  [-8.27915420e+00,  5.46828610e+00,  9.58443606e+00],  [-8.04580905e+00,  5.24142620e+00,  9.48546394e+00],  [-6.38957835e+00,  5.14330547e+00,  8.86970595e+00],  [-6.19781036e+00,  4.85517374e+00,  9.02680551e+00],  [-6.41512276e+00,  5.11818503e+00,  9.01518311e+00],  [-6.57795687e+00,  5.60585611e+00,  8.41095047e+00],  [-6.70674926e+00,  5.73764176e+00,  8.47391428e+00],  [-5.71895442e+00,  6.28836253e+00,  9.43531801e+00],  [-5.51367123e+00,  5.90598044e+00,  1.01947710e+01],  [-6.21874104e+00,  6.14059844e+00,  9.54496200e+00],  [-5.71751377e+00,  5.87234232e+00,  8.20035317e+00],  [-5.67996223e+00,  5.72174202e+00,  6.90297076e+00],  [-5.84022857e+00,  5.42607234e+00,  6.35729012e+00],  [-5.19315485e+00,  5.35680528e+00,  6.27151226e+00],  [-5.21749993e+00,  5.29016266e+00,  6.05342555e+00],  [-6.20663096e+00,  6.07405558e+00,  4.84483120e+00],  [-8.18690554e+00,  6.55495955e+00,  4.83453407e+00],  [-5.35835896e+00,  6.83065728e+00,  3.33440651e+00],  [-5.74226908e+00,  6.77919953e+00,  3.15341234e+00],  [-5.84409219e+00,  6.81273111e+00,  3.24670977e+00],  [-6.15474397e+00,  7.10149658e+00,  3.29064795e+00],  [-5.76827098e+00,  6.97367079e+00,  2.99213840e+00],  [-4.13162675e+00,  6.44609385e+00,  4.45075894e+00],  [-4.40491616e+00,  6.31752356e+00,  3.98943587e+00],  [-5.42938877e+00,  5.21611892e+00,  4.32860353e+00],  [-5.25806797e+00,  4.82668007e+00,  4.28831517e+00],  [-5.77343003e+00,  5.58896313e+00,  4.68357174e+00],  [-6.26034038e+00,  4.39715182e+00,  3.36337158e+00],  [-6.24802182e+00,  3.71351384e+00,  4.11292819e+00],  [-5.82437005e+00,  3.33614709e+00,  4.27922549e+00],  [-6.38977189e+00,  3.33012011e+00,  3.61873666e+00],  [-6.03879553e+00,  2.91551148e+00,  5.44701722e+00],  [-7.48157743e+00,  3.85938069e+00,  7.35986735e+00],  [-7.06590860e+00,  4.29111098e+00,  7.65478189e+00],  [-7.16888985e+00,  3.20494718e+00,  9.25813801e+00],  [-6.44643248e+00,  2.86364616e+00,  9.81724002e+00],  [-6.23847867e+00,  2.51570350e+00,  1.06696425e+01],  [-6.41749168e+00,  1.93422343e+00,  1.15502073e+01],  [-6.10273294e+00,  2.34102785e+00,  1.13397243e+01],  [-6.30546428e+00,  1.55529958e+00,  1.11646575e+01],  [-6.35970694e+00,  2.29018082e+00,  1.19185450e+01],  [-6.75830810e+00,  2.84313744e+00,  1.14362297e+01],  [-8.07345603e+00,  2.62677595e+00,  1.20533779e+01],  [-7.72724234e+00,  2.79939222e+00,  1.25114763e+01],  [-7.14010504e+00,  2.57004640e+00,  1.27387659e+01],  [-7.58677727e+00,  3.18040389e+00,  1.39951806e+01],  [-5.63710688e+00,  2.71739271e+00,  1.27009633e+01],  [-5.73657316e+00,  2.68452691e+00,  1.27493719e+01],  [-5.74830728e+00,  2.19547123e+00,  1.23960906e+01],  [-5.63838381e+00,  2.24458988e+00,  1.22804047e+01],  [-5.46996493e+00,  2.35602338e+00,  1.19329698e+01],  [-4.83900461e+00,  3.56846290e+00,  1.37223673e+01],  [-5.30411070e+00,  3.20706388e+00,  1.27683877e+01],  [-5.10953786e+00,  3.49733657e+00,  1.28244123e+01],  [-5.89918147e+00,  3.85905834e+00,  1.35897324e+01],  [-5.92878223e+00,  3.90618103e+00,  1.40827970e+01],  [-5.75057841e+00,  3.78234447e+00,  1.37391942e+01],  [-6.50125178e+00,  4.24200594e+00,  1.35533651e+01],  [-6.56918301e+00,  4.41160900e+00,  1.39986479e+01],  [-5.84906071e+00,  4.63992001e+00,  1.36894100e+01],  [-6.45400959e+00,  4.08731475e+00,  1.39586902e+01],  [-6.18208556e+00,  4.37495791e+00,  1.39684354e+01],  [-6.15895608e+00,  4.88342416e+00,  1.35041683e+01],  [-5.52092305e+00,  3.78629731e+00,  1.45724259e+01],  [-4.81158470e+00,  3.47719311e+00,  1.45488778e+01],  [-5.24619090e+00,  3.67615111e+00,  1.38366842e+01],  [-5.25363823e+00,  3.82440476e+00,  1.32557233e+01],  [-5.22805382e+00,  3.60646554e+00,  1.29241189e+01],  [-4.90612738e+00,  3.19363726e+00,  1.34706230e+01],  [-5.21508393e+00,  3.75433760e+00,  1.27343264e+01],  [-6.14392399e+00,  3.99626297e+00,  1.03536480e+01],  [-6.10049756e+00,  4.24142324e+00,  1.03275323e+01],  [-5.29103869e+00,  4.42321170e+00,  1.08115887e+01],  [-5.28785626e+00,  4.51065983e+00,  1.19292408e+01],  [-4.31774605e+00,  3.63026223e+00,  1.23086622e+01],  [-5.37371740e+00,  4.44888885e+00,  1.38388215e+01],  [-4.91093717e+00,  5.27637152e+00,  1.33047796e+01],  [-4.91913618e+00,  5.06899569e+00,  1.32667885e+01],  [-5.42571703e+00,  5.21728376e+00,  1.35062882e+01],  [-5.51582321e+00,  5.50296706e+00,  1.31385704e+01],  [-6.38864057e+00,  6.12816151e+00,  1.25933916e+01],  [-6.07772030e+00,  5.62146026e+00,  1.35106345e+01],  [-6.60069663e+00,  5.63653571e+00,  1.43053156e+01],  [-6.95327999e+00,  5.89581890e+00,  1.33926127e+01],  [-7.51027502e+00,  5.37085819e+00,  1.20230569e+01],  [-7.15723142e+00,  5.30015344e+00,  1.15660403e+01],  [-6.90021246e+00,  6.03577606e+00,  1.17851251e+01],  [-6.62816343e+00,  6.12962814e+00,  1.30271578e+01],  [-7.76538253e+00,  6.46320311e+00,  1.40728801e+01],  [-8.07715579e+00,  5.83474264e+00,  1.40669059e+01],  [-8.71721145e+00,  5.96642688e+00,  1.36049443e+01],  [-7.49501245e+00,  5.35412671e+00,  1.40715996e+01],  [-7.51771908e+00,  5.21244939e+00,  1.44118644e+01],  [-7.15571717e+00,  5.24038262e+00,  1.36723607e+01],  [-7.52125683e+00,  5.55538122e+00,  1.33288018e+01],  [-7.73402792e+00,  4.55273377e+00,  1.31752358e+01],  [-7.38048867e+00,  4.59620759e+00,  1.35952226e+01],  [-7.07980103e+00,  6.52877647e+00,  1.24797304e+01],  [-6.75757077e+00,  5.72304470e+00,  1.23637491e+01],  [-7.07984160e+00,  4.78074072e+00,  1.30725949e+01],  [-7.65233178e+00,  5.14380107e+00,  1.40580962e+01],  [-7.17874065e+00,  5.34807054e+00,  1.31414635e+01],  [-7.20819711e+00,  4.59462555e+00,  1.40496490e+01],  [-8.01090586e+00,  4.44948360e+00,  1.56310617e+01],  [-8.28022735e+00,  4.98819967e+00,  1.54084475e+01],  [-7.70412038e+00,  5.27400866e+00,  1.47243670e+01],  [-7.75729252e+00,  5.76453986e+00,  1.46378021e+01],  [-7.45899578e+00,  5.25112784e+00,  1.41463435e+01],  [-6.54346592e+00,  4.73143132e+00,  1.47156915e+01],  [-6.64966226e+00,  4.83265179e+00,  1.50373989e+01],  [-6.92729338e+00,  5.27039284e+00,  1.55241820e+01],  [-6.63467307e+00,  4.31697082e+00,  1.46145978e+01],  [-6.83027854e+00,  4.36090425e+00,  1.44031405e+01],  [-7.92792255e+00,  3.66790367e+00,  1.40379248e+01],  [-8.27612210e+00,  2.87522810e+00,  1.56483511e+01],  [-7.63635909e+00,  3.38843826e+00,  1.56439539e+01],  [-8.01212470e+00,  3.33017292e+00,  1.54189526e+01],  [-8.05296056e+00,  3.60121664e+00,  1.67911516e+01],  [-8.23123701e+00,  3.59242158e+00,  1.64004515e+01],  [-7.66873219e+00,  1.74548940e+00,  1.54180633e+01],  [-8.71576824e+00,  2.30467972e+00,  1.70652069e+01],  [-8.04975174e+00,  1.99919970e+00,  1.61698294e+01],  [-7.98122672e+00,  2.60854026e+00,  1.59590467e+01],  [-8.27827090e+00,  3.05679102e+00,  1.55663052e+01],  [-8.62184486e+00,  3.55378960e+00,  1.58796331e+01],  [-8.45287317e+00,  3.63673606e+00,  1.53277406e+01],  [-7.24219758e+00,  2.44426750e+00,  1.27031463e+01],  [-6.66666745e+00,  3.14224433e+00,  1.82614233e+01],  [-5.82309924e+00,  2.65155709e+00,  1.84304789e+01],  [-5.57110370e+00,  2.46703645e+00,  1.80726645e+01],  [-7.05914956e+00,  2.44902232e+00,  1.81959958e+01],  [-8.33525122e+00,  3.54555175e+00,  2.00090868e+01],  [-7.88060183e+00,  3.15126036e+00,  2.07891780e+01],  [-8.17646243e+00,  4.08730965e+00,  2.10998163e+01],  [-8.88195391e+00,  4.49864225e+00,  2.07950002e+01],  [-9.21053278e+00,  4.60272034e+00,  2.02768646e+01],  [-8.95912205e+00,  4.67321068e+00,  2.03458097e+01],  [-9.03132889e+00,  5.28435678e+00,  2.04717165e+01],  [-7.98770418e+00,  5.58785770e+00,  1.99579970e+01],  [-7.20013048e+00,  5.81925231e+00,  1.97074891e+01],  [-7.31484569e+00,  5.90827597e+00,  1.94021641e+01],  [-8.27061494e+00,  6.50625214e+00,  1.91923073e+01],  [-9.22794820e+00,  6.96720066e+00,  2.03447888e+01],  [-9.81674099e+00,  8.37520153e+00,  2.01449876e+01],  [-9.93510267e+00,  8.16639552e+00,  1.91606097e+01],  [-1.00746064e+01,  8.90134270e+00,  1.93402171e+01],  [-1.01410656e+01,  9.54110116e+00,  1.89084564e+01],  [-9.41289443e+00,  1.01256044e+01,  1.95781141e+01],  [-8.75756381e+00,  1.00501785e+01,  1.96189614e+01],  [-8.74368293e+00,  9.56153708e+00,  2.00718843e+01],  [-7.81839897e+00,  1.00638955e+01,  2.05163519e+01],  [-6.96056774e+00,  1.05791106e+01,  2.08022624e+01],  [-6.79093742e+00,  9.50084646e+00,  1.93404013e+01],  [-6.34396815e+00,  8.69908853e+00,  2.33889104e+01],  [-5.16354963e+00,  8.81359755e+00,  2.36579998e+01],  [-4.84962970e+00,  8.61139441e+00,  2.47841006e+01],  [-5.13466900e+00,  8.82128441e+00,  2.44052678e+01],  [-5.16312923e+00,  8.96569075e+00,  2.24692282e+01],  [-4.15895403e+00,  9.90400557e+00,  2.43044483e+01],  [-4.72719511e+00,  9.88409511e+00,  2.41983724e+01],  [-5.68550584e+00,  9.70165979e+00,  2.48614924e+01],  [-6.22025006e+00,  9.44463443e+00,  2.58844562e+01],  [-5.37775706e+00,  9.17891924e+00,  2.51299114e+01],  [-7.14925196e+00,  9.41721561e+00,  2.64937038e+01],  [-9.26392289e+00,  8.12628790e+00,  2.82781098e+01],  [-1.05613020e+01,  8.13733962e+00,  2.74653546e+01],  [-9.94439310e+00,  9.27047371e+00,  2.74069637e+01],  [-9.78936587e+00,  9.34977076e+00,  2.77121388e+01],  [-9.40110421e+00,  8.84680248e+00,  2.77137384e+01],  [-8.06537950e+00,  8.90423362e+00,  2.61650191e+01],  [-7.23797971e+00,  8.18112802e+00,  2.65008259e+01],  [-5.85212767e+00,  8.06786455e+00,  2.54142223e+01],  [-5.98128152e+00,  8.60293367e+00,  2.51382316e+01],  [-4.80267170e+00,  9.75780922e+00,  2.29811551e+01],  [-4.47034791e+00,  9.22158458e+00,  2.24527902e+01],  [-3.39924127e+00,  9.48804699e+00,  2.42031502e+01],  [-3.89787349e+00,  9.63624300e+00,  2.60988773e+01]], dtype=np.float64))
assert all((np.allclose(x, y, atol=1e-6) for x, y in zip(ret, target)))
print('Tests passed.')